In [13]:
%pip install pennylane scikit-learn numpy matplotlib
%pip install qiskit
%pip install qiskit[visualization] 
%pip install qiskit-aer
%pip install qiskit-ibm-runtime
%pip install qiskit-algorithms
%pip install qiskit-machine-learning
%pip install pylatexenc

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ---------------------------------------- 0.0/6.5 MB ? eta -:--:--
   ------------ --------------------------- 2.1/6.5 MB 11.2 MB/s eta 0:00:01
   ------------------------- -------------- 4.2/6.5 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------- 6.5/6.5 MB 10.6 MB/s  0:00:00
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   -- ------------------------------------- 2.1/41.0 MB 11.8 MB/s eta 0:00:04
   ---- ----------------------------------- 4.5/41.0 MB 11.7 MB/s eta 0:00:04
   ------ ---

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

DATASETS = {
    "concrete_lof": {
        "path": "concrete_cleanLOF.csv",
        "features": ["Cement","BlastFurnaceSlag","FlyAsh","Water","Superplasticizer","CoarseAggregate","FineAggregate","Age"],
        "target": "ConcreteCompressiveStrength",
        "outlier_flag": "is_outlier",
        "rename": None,          
        "scaling": {"standard": True, "minmax": (-1.0, 1.0)}
    },
    "baseball": {
        "path": "baseball_cleanLOF.csv",
        "features": ["Batting_average","On-base_percentage","Runs","Hits","Doubles","Triples","HomeRuns","Runs_batted_in","Walks","Strike-Outs","Stolen_bases","Errors","Free_agency_eligibility","Free_agent","Arbitration_eligibility","Arbitration","Salary"],
        "target": None,          
        "outlier_flag": "is_outlier",
        "rename": None,
        "scaling": {"standard": True, "minmax": (-1.0, 1.0)}
    },
    "concrete_ocsvm": {
        "path": "concrete_cleanOCSVM.csv",
        "features": ["Cement","BlastFurnaceSlag","FlyAsh","Water","Superplasticizer","CoarseAggregate","FineAggregate","Age"],
        "target": "ConcreteCompressiveStrength",
        "outlier_flag": "ocsvm_outlier",
        "rename": None,
        "scaling": {"standard": True, "minmax": (-1.0, 1.0)}
    },
    "elevators": {
        "path": "Elevators_cleanLOF.csv",
        "features": ["ClimbRate","Sgz","P","Q","CurRoll","AbsRoll","DiffClb","DiffRollRate","DiffDiffClb","SaTime1","SaTime2","SaTime3","SaTime4","DiffSaTime1","DiffSaTime2","DiffSaTime3","DiffSaTime4","Sa","Goal"],
        "target": "Goal",
        "outlier_flag": "is_outlier",
        "rename": None,
        "scaling": {"standard": True, "minmax": (-1.0, 1.0)}
    },
}

def load_dataset(spec_name: str):
    spec = DATASETS[spec_name]
    path = Path(spec["path"])
    if not path.exists():
        raise FileNotFoundError(f"No se encontró {path.resolve()}")
    df = pd.read_csv(path)
    if spec.get("rename"):
        df = df.rename(columns=spec["rename"])
    feats = spec["features"]
    missing = [c for c in feats if c not in df.columns]
    if missing:
        raise ValueError(f"Faltan columnas en el CSV: {missing}")
    X = df[feats].copy()
    imputer = SimpleImputer(strategy="median")
    X = pd.DataFrame(imputer.fit_transform(X), columns=feats)

    if spec["scaling"].get("standard"):
        X = pd.DataFrame(StandardScaler().fit_transform(X), columns=feats)
    if spec["scaling"].get("minmax"):
        X = pd.DataFrame(MinMaxScaler(feature_range=spec["scaling"]["minmax"]).fit_transform(X), columns=feats)

    X = X.to_numpy(dtype=np.float64)

    tgt = spec.get("target")
    flag = spec.get("outlier_flag")

    meta = {
        "name": spec_name,
        "path": str(path),
        "features": feats,
        "target": tgt,
        "outlier_flag": flag,
        "N": X.shape[0],
        "d": X.shape[1],
    }
    return X, meta

In [5]:
DATASET_NAME = "concrete_lof"  # "concrete_lof" | "baseball" | "concrete_ocsvm" | "elevators"
X, meta = load_dataset(DATASET_NAME)
print(meta)

{'name': 'concrete_lof', 'path': 'concrete_cleanLOF.csv', 'features': ['Cement', 'BlastFurnaceSlag', 'FlyAsh', 'Water', 'Superplasticizer', 'CoarseAggregate', 'FineAggregate', 'Age'], 'target': 'ConcreteCompressiveStrength', 'outlier_flag': 'is_outlier', 'N': 978, 'd': 8}


In [6]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

assert 'X' in globals()
N, d = X.shape

def _normalize_pad_pow2(v):
    v = np.array(v, dtype=float).ravel()
    n = int(np.ceil(np.log2(max(1, len(v)))))
    size = 2**n
    if len(v) < size:
        v = np.pad(v, (0, size - len(v)))
    norm = np.linalg.norm(v)
    if norm == 0:
        v = np.zeros_like(v); v[0] = 1.0
    else:
        v = v / norm
    return v

def qc_angle(x):
    qc = QuantumCircuit(d)
    for i, xi in enumerate(x):
        qc.ry(float(xi), i)
    return qc

def qc_basis(x):
    qc = QuantumCircuit(d)
    bits = (x > 0).astype(int)
    for i, bi in enumerate(bits):
        if int(bi) == 1:
            qc.x(i)
    return qc

def qc_amplitude(x):
    n = int(np.ceil(np.log2(d)))
    qc = QuantumCircuit(n)
    v = _normalize_pad_pow2(x)
    psi = v.astype(complex)
    qc.initialize(psi, list(range(n)))
    return qc

def sv_angle(x):
    return Statevector.from_instruction(qc_angle(x))

def sv_basis(x):
    return Statevector.from_instruction(qc_basis(x))

def sv_amplitude(x):
    return Statevector.from_instruction(qc_amplitude(x))

In [7]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import QFT, RZZGate, RXXGate

assert 'X' in globals()
N, d = X.shape

def qc_fourier_map(x):
    qc = QuantumCircuit(d)
    for i, xi in enumerate(x):
        qc.rz(float(xi), i)
    qc.compose(QFT(d, do_swaps=True), inplace=True)
    return qc

def qc_zz_feature_map(x):
    qc = QuantumCircuit(d)
    for i, xi in enumerate(x):
        qc.rz(float(xi), i)
    for i in range(d):
        for j in range(i+1, d):
            qc.append(RZZGate(2.0*float(x[i%len(x)])*float(x[j%len(x)])), [i, j])
    return qc

def qc_pauli_feature_map(x):
    qc = QuantumCircuit(d)
    for i, xi in enumerate(x):
        qc.rx(float(xi), i)
        qc.rz(float(xi), i)
    for i in range(d):
        for j in range(i+1, d):
            qc.append(RXXGate(2.0*float(x[i%len(x)])*float(x[j%len(x)])), [i, j])
            qc.append(RZZGate(2.0*float(x[i%len(x)])*float(x[j%len(x)])), [i, j])
    return qc

def sv_fourier(x):
    return Statevector.from_instruction(qc_fourier_map(x))

def sv_zz(x):
    return Statevector.from_instruction(qc_zz_feature_map(x))

def sv_pauli(x):
    return Statevector.from_instruction(qc_pauli_feature_map(x))

In [8]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def load_target(meta):
    if meta["target"] is None:
        raise ValueError("Este dataset no tiene columna target definida en DATASETS.")
    df = pd.read_csv(meta["path"])
    if meta["target"] not in df.columns:
        raise ValueError(f"No existe la columna target '{meta['target']}' en el CSV.")
    y = df[meta["target"]].to_numpy(dtype=np.float64).ravel()
    return y

def print_regression_report(y_true, y_pred, label=""):
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f"[{label}] R2: {r2:.4f} | MSE: {mse:.4f} | MAE: {mae:.4f}")

y = load_target(meta)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((782, 8), (196, 8), (782,), (196,))

In [ ]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms.regressors import QSVR

fm = ZZFeatureMap(feature_dimension=X.shape[1], reps=2, entanglement="full")

qkernel = FidelityQuantumKernel(feature_map=fm)

qsvr = QSVR(quantum_kernel=qkernel, C=3.0, epsilon=0.1)

qsvr.fit(X_train, y_train)
y_pred = qsvr.predict(X_test)

print_regression_report(y_test, y_pred, label="QSVR + ZZFeatureMap")

In [ ]:
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import SLSQP
from qiskit_machine_learning.algorithms.regressors import VQR

fm = ZZFeatureMap(feature_dimension=X.shape[1], reps=2, entanglement="full")

ansatz = RealAmplitudes(num_qubits=X.shape[1], reps=2, entanglement="full")

opt = SLSQP(maxiter=200)

estimator = Estimator()

vqr = VQR(
    feature_map=fm,
    ansatz=ansatz,
    optimizer=opt,
    estimator=estimator,
    loss="squared_error",
    warm_start=False
)

vqr.fit(X_train, y_train)
y_pred_vqr = vqr.predict(X_test)

print_regression_report(y_test, y_pred_vqr, label="VQR (ZZFeatureMap + RealAmplitudes)")

C:\Users\Admin\AppData\Local\Temp\ipykernel_15012\1232141114.py:12: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


In [ ]:
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor
from qiskit_algorithms.optimizers import SLSQP

num_qubits = X.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2)
ansatz = RealAmplitudes(num_qubits=num_qubits, reps=2, entanglement="full")

from qiskit import QuantumCircuit
qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

estimator = Estimator()
qnn = EstimatorQNN(
    estimator=estimator,
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters
)

regressor = NeuralNetworkRegressor(
    neural_network=qnn,
    optimizer=SLSQP(maxiter=200),
    loss='squared_error'
)

regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
print_regression_report(y_test, y_pred, label="NeuralNetworkRegressor + ZZFeatureMap")
